In [ ]:
import cv2
import time
import numpy as np
import os
import mediapipe as mp
import time
import tensorflow as tf
from collections import deque
import pyttsx3

In [ ]:
STATIC_LABELS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 
                 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 
                 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
DYNAMIC_LABELS = ['AKU', 'HALO', 'APA', 'KABAR', 'KAMU', 'NAMA', 'PERKENALKAN', 
                  'SIAPA', 'TERIMA KASIH', 'TOLONG', 'SALAM KENAL', 'MAAF', 
                  'BERASAL', 'DARI', 'MANA', 'SAMA-SAMA', 'YA', 'TIDAK', 'JAKARTA']

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def extract_keypoints(results, include_pose=True):
    pose = [[res.x, res.y] for res in results.pose_landmarks.landmark] if results.pose_landmarks else [[0, 0]] * 33
    lh = [[res.x, res.y] for res in results.left_hand_landmarks.landmark] if results.left_hand_landmarks else [[0, 0]] * 21
    rh = [[res.x, res.y] for res in results.right_hand_landmarks.landmark] if results.right_hand_landmarks else [[0, 0]] * 21

    if include_pose:
        return np.array(pose + lh + rh).flatten()
    else:
        return np.array(lh + rh).flatten()

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = holistic.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
def draw_landmarks(image, results, include_pose=True):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    if include_pose:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

In [ ]:
def normalize_keypoints(keypoints):
    keypoints = np.array(keypoints).reshape(-1, 2)
    min_val = keypoints.min(axis=0)
    max_val = keypoints.max(axis=0)
    range_val = np.where((max_val - min_val) == 0, 1, (max_val - min_val))
    normed = (keypoints - min_val) / range_val
    return normed.flatten()

In [ ]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [ ]:
def compute_movement(curr, prev):
    if prev is None:
        return 0.0
    return np.linalg.norm(curr - prev)

In [ ]:
model_static = tf.keras.models.load_model("static_model.h5")
model_dynamic = tf.keras.models.load_model("high_dynamic_model.h5")

In [ ]:
SEQUENCE_LENGTH = 30
MOVEMENT_THRESHOLD = 0.04
CONFIDENCE_THRESHOLD = 0.9
silence_timeout = 2.0